<a href="https://colab.research.google.com/github/andrybrew/all-files/blob/master/003_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification

In [0]:
# Import Library
import pandas as pd

#Import the files to Google Colab
url = 'https://raw.githubusercontent.com/andrybrew/all-files/master/dataset_python/churn_transformed.csv'
df_csv = pd.read_csv(url, sep=',')

# Show 10 first Row
df_csv.head()

In [0]:
# Remove "Unnamed:O" Coloumn
df = df_csv.drop("Unnamed: 0", axis=1)
df.head()

In [0]:
# Check the Data Infomation
df.info()

In [0]:
#Import MinMax Scaler
from sklearn.preprocessing import MinMaxScaler

# initialize min-max scaler
mm_scaler = MinMaxScaler()
column_names = df.columns.tolist()
column_names.remove('Churn')

# Transform all attributes
df[column_names] = mm_scaler.fit_transform(df[column_names])
df.sort_index(inplace=True)
df.head()

In [0]:
# Selecting the Feature, by remove the unused feature 
feature = ['Churn', 'TotalCharges']
train_feature = df.drop(feature, axis=1)

# Set The Target
train_target = df["Churn"]

In [0]:
# Show the Feature
train_feature.head(5)

In [0]:
# Split Data
from sklearn.model_selection import train_test_split, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(train_feature ,train_target, shuffle = True, test_size=0.3, random_state=1)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [0]:
# Show the training data
X_train.head()

#### Decision Tree

We use [Scikit Learn DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) function. Below is the default parameter:


`DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)`,

An explanation of the decision tree can be seen here.[Medium: Decision Tree](https://medium.com/deep-math-machine-learning-ai/chapter-4-decision-trees-algorithms-b93975f7a1f1)

In [0]:
# Import library
from sklearn import tree

# Train Decision Tree
dtc = tree.DecisionTreeClassifier(min_impurity_decrease=0.01)
dtc.fit(X_train, y_train)

# Predict to Test Data 
y_preddtc = dtc.predict(X_test)

In [0]:
# Plot the tree
tree.plot_tree(dtc.fit(X_train, y_train), class_names=['0','1']) 

In [0]:
# Visualize with graphviz

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(dtc, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [0]:
# Import the metrics class
from sklearn import metrics

# Show the Confussion Matrix
cnf_matrixdtc = metrics.confusion_matrix(y_test, y_preddtc)
cnf_matrixdtc

In [0]:
# Show the Accuracy, Precision, Recall
acc_dtc = metrics.accuracy_score(y_test, y_preddtc)
prec_dtc = metrics.precision_score(y_test, y_preddtc)
rec_dtc = metrics.recall_score(y_test, y_preddtc)
f1_dtc = metrics.f1_score(y_test, y_preddtc)
kappa_dtc = metrics.cohen_kappa_score(y_test, y_preddtc)

print("Accuracy:", acc_dtc )
print("Precision:", prec_dtc)
print("Recall:", rec_dtc)
print("F1 Score:", f1_dtc)
print("Cohens Kappa Score:", kappa_dtc)

In [0]:
# Cross-validation score
cv_iterations = 5
cv_score = cross_val_score(dtc, train_feature, train_target, cv=cv_iterations)
print('Accuracy with cross-validation (split size = {}): {} (+/- {})'
      .format(cv_iterations, round(cv_score.mean(),2), round(cv_score.std() * 2,2)))

In [0]:
# Import Visualization Package
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (7, 7)
plt.style.use('ggplot')

# Visualize ROC Curve
y_pred_probadtc = dtc.predict_proba(X_test)[::,1]
fprdtc, tprdtc, _ = metrics.roc_curve(y_test,  y_pred_probadtc)
aucdtc = metrics.roc_auc_score(y_test, y_pred_probadtc)
plt.plot(fprdtc,tprdtc,label="Decision Tree, auc="+str(aucdtc))
plt.legend(loc=4)
plt.show()

#### K-Nearest Neighbor

We use Scikit Learn KNeighborsClassifier function. Here is the default parameter:

`KNeighborsClassifier(n_neighbors=5, weights=’uniform’, algorithm=’auto’, leaf_size=30, p=2, metric=’minkowski’, metric_params=None, n_jobs=None, **kwargs)`

A small value of k means that noise will have a higher influence on the result and a large value make it computationally expensive. Data scientists usually choose as an odd number if the number of classes is 2 and another simple approach to select k is set k=sqrt(n).

In [0]:
# Import library
from sklearn.neighbors import KNeighborsClassifier

# Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors= 75)
knn.fit(X_train, y_train)

# Predict to test data
y_predknn = knn.predict(X_test)

In [0]:
# Show the Confussion Matrix
cnf_matrixknn = metrics.confusion_matrix(y_test, y_predknn)
cnf_matrixknn

In [0]:
# Show the Accuracy, Precision, Recall
acc_knn = metrics.accuracy_score(y_test, y_predknn)
prec_knn = metrics.precision_score(y_test, y_predknn)
rec_knn = metrics.recall_score(y_test, y_predknn)
f1_knn = metrics.f1_score(y_test, y_predknn)
kappa_knn = metrics.cohen_kappa_score(y_test, y_predknn)

print("Accuracy:", acc_knn)
print("Precision:", prec_knn)
print("Recall:", rec_knn)
print("F1 Score:", f1_knn)
print("Cohens Kappa Score:", kappa_knn)

In [0]:
# Cross-validation score
cv_iterations = 10
cv_score = cross_val_score(knn, train_feature, train_target, cv=cv_iterations)
print('Accuracy with cross-validation (split size = {}): {} (+/- {})'
      .format(cv_iterations, round(cv_score.mean(),2), round(cv_score.std() * 2,2)))

In [0]:
# ROC Curve
y_pred_probaknn = knn.predict_proba(X_test)[::,1]
fprknn, tprknn, _ = metrics.roc_curve(y_test,  y_pred_probaknn)
aucknn = metrics.roc_auc_score(y_test, y_pred_probaknn)
plt.plot(fprknn,tprknn,label="K-NN, auc="+str(aucknn))
plt.legend(loc=4)
plt.show()

#### Naive Bayes

We use Scikit Learn [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html) function. Here is the default parameter:

`class sklearn.naive_bayes.GaussianNB(priors=None, var_smoothing=1e-09`


In [0]:
from sklearn.naive_bayes import GaussianNB 

# Train Naive Bayes Model
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_predgnb= gnb.predict(X_test)

In [0]:
# Show the Confussion Matrix
cnf_matrixgnb = metrics.confusion_matrix(y_test, y_predgnb)
cnf_matrixgnb

In [0]:
# Show the Accuracy, Precision, Recall
acc_gnb = metrics.accuracy_score(y_test, y_predgnb)
prec_gnb = metrics.precision_score(y_test, y_predgnb)
rec_gnb = metrics.recall_score(y_test, y_predgnb)
f1_gnb = metrics.f1_score(y_test, y_predgnb)
kappa_gnb = metrics.cohen_kappa_score(y_test, y_predgnb)

print("Accuracy:", acc_gnb)
print("Precision:", prec_gnb)
print("Recall:", rec_gnb)
print("F1 Score:", f1_gnb)
print("Cohens Kappa Score:", kappa_gnb)

In [0]:
# Cross-validation score
cv_iterations = 10
cv_score = cross_val_score(gnb, train_feature, train_target, cv=cv_iterations)
print('Accuracy with cross-validation (split size = {}): {} (+/- {})'
      .format(cv_iterations, round(cv_score.mean(),2), round(cv_score.std() * 2,2)))

In [0]:
# ROC Curve
y_pred_probagnb = gnb.predict_proba(X_test)[::,1]
fprgnb, tprgnb, _ = metrics.roc_curve(y_test,  y_pred_probagnb)
aucgnb = metrics.roc_auc_score(y_test, y_pred_probagnb)
plt.plot(fprgnb,tprgnb,label="Naive Bayes, auc="+str(aucgnb))
plt.legend(loc=4)
plt.show()

#### Model Comparison

In [0]:
# Comparing Model Performance
print("Decision Tree Accuracy =",acc_dtc)
print("Decision Tree Precision =",prec_dtc)
print("Decision Tree Recall =",rec_dtc)
print("Decision Tree F1-Score =", f1_dtc)
print("_______________________")
print("k-NN Accuracy =", acc_knn)
print("k-NN Precision =", prec_knn)
print("k-NN Recall =", rec_knn)
print("k-NN F1-Score =", f1_knn)
print("_______________________")
print("Naive Bayes Accuracy =", acc_gnb)
print("Naive Bayes Precision =", prec_gnb)
print("Naive Bayes Recall =", rec_gnb)
print("Naive Bayes F1-Score =", f1_gnb)

In [0]:
# Comparing ROC Curve
plt.plot(fprdtc,tprdtc,label="Decision Tree, auc="+str(aucdtc))
plt.plot(fprknn,tprknn,label="K-NN, auc="+str(aucknn))
plt.plot(fprgnb,tprgnb,label="Naive Bayes, auc="+str(aucgnb))
plt.legend(loc=4)
plt.show()